In [1]:
import pandas as pd

### Create inverse rates

In [48]:
df = pd.read_csv('testData.csv')
# append copy of dataframe to create inverse rates
df2 = df.copy()
df2.Rate = list(map(lambda x: float(1/x), df2.Rate))
df2 = df2.rename(columns={'Numerator':'Denumerator','Denumerator':'Numerator'})

df = (df
        .append(df2)
        .drop(columns=['Unnamed: 0'])
        .reset_index()
        .drop(columns=['index'])
     )

### Create possible pairs in triangular arbitrage

In [10]:
pairList = []
initialCoin = 'BTC'
# for every denumerator that has BTC as numerator
for i in list(df.Denumerator[df['Numerator'] == initialCoin]):
    # for every denumerator that has i as numerator
    for j in list(set((df.Denumerator[(df['Numerator'] == i) & (df['Numerator'] != initialCoin) & (df['Denumerator'] != initialCoin)]))):
        if j in list(df.Denumerator[df['Numerator'] == initialCoin]):
            pairList.append([i,j])
len(pairList)

120

### Data preparation function

In [53]:
def prepareData(df, initialCoin = 'BTC'):
    # INVERSE RATES CREATION
    # append copy of dataframe to create inverse rates
    df2 = df.copy()
    df2.Rate = list(map(lambda x: float(1/x), df2.Rate))
    df2 = df2.rename(columns={'Numerator':'Denumerator','Denumerator':'Numerator'})

    df = (df
            .append(df2)
            .drop(columns=['Unnamed: 0'])
            .reset_index()
            .drop(columns=['index'])
         )
    
    # POSSIBLE PAIRS CREATION
    pairList = []
    # for every denumerator that has BTC as numerator
    for i in list(df.Denumerator[df['Numerator'] == initialCoin]):
        # for every denumerator that has i as numerator
        for j in list(set((df.Denumerator[(df['Numerator'] == i) & (df['Numerator'] != initialCoin) & (df['Denumerator'] != initialCoin)]))):
            if j in list(df.Denumerator[df['Numerator'] == initialCoin]):
                pairList.append([i,j])
    
    return df, pairList

### Real data arbitrage

In [54]:
def triangularArbitrage(inputSet, firstTransaction, secondTransaction, fee):
    # first transaction
    output1 = inputSet[0]/df.Rate.loc[(df['Numerator'] == inputSet[1]) & (df['Denumerator'] == firstTransaction)].values[0]
    output1 = output1 - fee*output1
    
    # second transaction
    output2 = output1/df.Rate.loc[(df['Numerator'] == firstTransaction) & (df['Denumerator'] == secondTransaction)].values[0]
    output2 = output2 - fee*output2
    
    # third transaction
    output3 = output2/df.Rate.loc[(df['Numerator'] == secondTransaction) & (df['Denumerator'] == inputSet[1])].values[0]
    output2 = output2 - fee*output2
    
#     if float(output3 - inputSet[0]) > 0:
#         print(f'Obtained by {inputSet[1]} > {firstTransaction} > {secondTransaction} > {inputSet[1]}:', float(output3 - inputSet[0]), inputSet[1])
    
    return float(output3 - inputSet[0])

# inputSet = (1, 'BTC')
# %time triangularArbitrage(inputSet, 'LTC', 'ETH', fee = 0.0004)

In [55]:
# read data
df = pd.read_csv('testData.csv')
# transform dataframe
initialCoin = 'BTC'
df, pairList = prepareData(df, initialCoin = initialCoin)
print('Possible initial coins:', list(df[:103].Denumerator.unique()))

Possible initial coins: ['USDT', 'BTC', 'PAX', 'ETH', 'USDC']


In [56]:
%%time
suma = 0
n = 0
btcQuantity = 1
for i in pairList:
    add = triangularArbitrage((btcQuantity, initialCoin), i[0], i[1], fee = 0.0004)
    if add > 0:
        suma += add
        btcQuantity += add
        n += 1
print(f'Obtained: {round(suma * 20143.63, 2)} PLN by {n} triangular transactions using cummulated {initialCoin} asset')

Obtained: 5781.32 PLN by 44 triangular transactions using cummulated BTC asset
Wall time: 363 ms
